## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dwarka,IN,22.2394,68.9678,70.00,61,0,12.50,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,62.60,34,0,12.66,clear sky
2,2,Pangoa,PE,-12.1167,-73.0000,77.18,89,100,1.52,overcast clouds
3,3,Mar Del Plata,AR,-38.0023,-57.5575,64.40,82,90,8.05,moderate rain
4,4,Noumea,NC,-22.2763,166.4572,78.01,94,0,2.30,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip?"))
max_temp= float(input("What is the minimum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?45
What is the minimum temperature you would like for your trip?75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dwarka,IN,22.2394,68.9678,70.00,61,0,12.50,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,62.60,34,0,12.66,clear sky
3,3,Mar Del Plata,AR,-38.0023,-57.5575,64.40,82,90,8.05,moderate rain
7,7,Mataura,NZ,-46.1927,168.8643,52.43,88,99,1.88,overcast clouds
8,8,Gelemso,ET,8.8167,40.5167,63.93,66,24,8.10,few clouds
10,10,Hilo,US,19.7297,-155.0900,64.40,59,20,8.05,few clouds
13,13,Ushuaia,AR,-54.8000,-68.3000,59.00,55,0,13.80,clear sky
19,19,Port Lincoln,AU,-34.7333,135.8667,59.50,80,14,16.46,few clouds
20,20,Busselton,AU,-33.6500,115.3333,63.00,98,100,1.01,overcast clouds
26,26,Kapaa,US,22.0752,-159.3190,60.80,72,1,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 0 to 692
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              176 non-null    int64  
 1   City                 176 non-null    object 
 2   Country              174 non-null    object 
 3   Lat                  176 non-null    float64
 4   Lng                  176 non-null    float64
 5   Max Temp             176 non-null    float64
 6   Humidity             176 non-null    int64  
 7   Cloudiness           176 non-null    int64  
 8   Wind Speed           176 non-null    float64
 9   Current Description  176 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.1+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
final_cities_df = preferred_cities_df.dropna()
final_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 692
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              174 non-null    int64  
 1   City                 174 non-null    object 
 2   Country              174 non-null    object 
 3   Lat                  174 non-null    float64
 4   Lng                  174 non-null    float64
 5   Max Temp             174 non-null    float64
 6   Humidity             174 non-null    int64  
 7   Cloudiness           174 non-null    int64  
 8   Wind Speed           174 non-null    float64
 9   Current Description  174 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 15.0+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = final_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dwarka,IN,70.00,clear sky,22.2394,68.9678,
1,Punta Arenas,CL,62.60,clear sky,-53.1500,-70.9167,
3,Mar Del Plata,AR,64.40,moderate rain,-38.0023,-57.5575,
7,Mataura,NZ,52.43,overcast clouds,-46.1927,168.8643,
8,Gelemso,ET,63.93,few clouds,8.8167,40.5167,
10,Hilo,US,64.40,few clouds,19.7297,-155.0900,
13,Ushuaia,AR,59.00,clear sky,-54.8000,-68.3000,
19,Port Lincoln,AU,59.50,few clouds,-34.7333,135.8667,
20,Busselton,AU,63.00,overcast clouds,-33.6500,115.3333,
26,Kapaa,US,60.80,clear sky,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 

    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "",:]

In [10]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Dwarka,IN,70.00,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
1,Punta Arenas,CL,62.60,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
3,Mar Del Plata,AR,64.40,moderate rain,-38.0023,-57.5575,Gran Hotel Mar del Plata
7,Mataura,NZ,52.43,overcast clouds,-46.1927,168.8643,Ellie's Villa
8,Gelemso,ET,63.93,few clouds,8.8167,40.5167,Kerod Hotel


In [11]:
# 8a. Create the output File (CSV)

output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 

max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))